In [1]:
import numpy as np
import pandas as pd
import boto3

In [36]:
ACCESS_KEY = 
SECRET_KEY = 

s3 = boto3.resource('s3',
         aws_access_key_id=ACCESS_KEY,
         aws_secret_access_key=SECRET_KEY)

s3bucket = s3.Bucket('personalityfromphoto')

In [26]:
photolist = list()
for i in s3bucket.objects.all():
    photolist.append(i.key)

allphotos = pd.DataFrame(photolist,columns=['fileName'])

In [27]:
len(photolist)

26689

In [28]:
photolist[:10]

['E/1000006876806379.jpg',
 'E/1001091583356351.jpg',
 'E/1001328126675771.jpg',
 'E/1002119396597737.jpg',
 'E/1003023426504245.jpg',
 'E/1003994419776956.jpg',
 'E/1004190739722068.jpg',
 'E/1004511343038371.jpg',
 'E/1004532636353049.jpg',
 'E/1005244356283439.jpg']

In [29]:
allphotos.head()

,fileName
0,E/1000006876806379.jpg
1,E/1001091583356351.jpg
2,E/1001328126675771.jpg
3,E/1002119396597737.jpg
4,E/1003023426504245.jpg


In [30]:
allphotos['fileName'][1]

'E/1001091583356351.jpg'

In [38]:
bucket='personalityfromphoto'

client=boto3.client('rekognition', 
                    aws_access_key_id=ACCESS_KEY, 
                    aws_secret_access_key=SECRET_KEY, 
                    region_name='us-west-1')

Labels = []
for i in range(10):
    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':allphotos.fileName[i]}},MinConfidence=50,MaxLabels=50)
    Labels.append(response['Labels'])
    
xPhotos = photolist[0:10]

pilot = pd.DataFrame(xPhotos,columns=['fileName'])
pilot['Labels'] = Labels

EndpointConnectionError: Could not connect to the endpoint URL: "https://rekognition.us-west-1.amazonaws.com/"

In [32]:
response1 = client.detect_faces(Image={
        'S3Object':{
            'Bucket':bucket,
            'Name':'1001091583356351.jpg'}},
                                    Attributes=['ALL'])

InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the DetectFaces operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

In [33]:
faceDetails = []
for i in range(10):  
    response1 = client.detect_faces(Image={
        'S3Object':{
            'Bucket':bucket,
            'Name':allphotos['fileName'][i]}},
                                    Attributes=['ALL'])
    faceDetails.append(response1['FaceDetails'])

InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the DetectFaces operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

In [17]:

BUCKET = 'personalityfromphoto'
KEY = 'test.jpg'
FEATURES_BLACKLIST = ("Landmarks", "Emotions", "Pose", "Quality", "BoundingBox", "Confidence")

def detect_faces(bucket, key, attributes=['ALL'], region='us-west-2'):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.detect_faces(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        Attributes=attributes,
    )
    return response['FaceDetails']


In [18]:
for face in detect_faces(BUCKET, KEY):
    print('Face ({Confidence}%)'.format(**face))
    # emotions
    for emotion in face['Emotions']:
        print("  {Type} : {Confidence}%".format(**emotion))
    # quality
    for quality, value in face['Quality'].iteritems():
        print("  {quality} : {value}".format(quality=quality, value=value))
    # facial features
    for feature, data in face.iteritems():
        if feature not in FEATURES_BLACKLIST:
            print("  {feature}({data[Value]}) : {data[Confidence]}%".format(feature=feature, data=data))


NoCredentialsError: Unable to locate credentials